## Import Libraries

In [25]:
# Cell 1: Import Libraries
import pandas as pd
import os
import datetime as dt
print("Libraries imported successfully.")

Libraries imported successfully.


## Load Cleaned Data

In [26]:
# Cell 2: Load Cleaned Data
# Define the path to your cleaned data file in the 'data' folder
data_folder = '../data' # Relative path from notebooks to data folder
cleaned_file_name = 'online_retail_dataset_preprocessing.xlsx'
cleaned_file_path = os.path.join(data_folder, cleaned_file_name)

print(f"Loading cleaned data from: {cleaned_file_path}")

# Load the cleaned DataFrame
df_cleaned = pd.read_excel(cleaned_file_path)

print(f"Cleaned data loaded. Shape: {df_cleaned.shape}")
print("First 5 rows of cleaned data (with correct TotalPrice):")
display(df_cleaned.head())

Loading cleaned data from: ../data\online_retail_dataset_preprocessing.xlsx
Cleaned data loaded. Shape: (779425, 9)
First 5 rows of cleaned data (with correct TotalPrice):


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


## Determine Snapshot Date for RFM Analysis

In [27]:
# Cell 3: Determine Snapshot Date for RFM Analysis
# To calculate Recency, we need a "snapshot date" or "analysis date".
# This is typically one day after the last transaction date in the dataset.
snapshot_date = df_cleaned['InvoiceDate'].max() + dt.timedelta(days=1)

print(f"The latest invoice date in the dataset is: {df_cleaned['InvoiceDate'].max()}")
print(f"The RFM analysis snapshot date is: {snapshot_date}")

The latest invoice date in the dataset is: 2011-12-09 12:50:00
The RFM analysis snapshot date is: 2011-12-10 12:50:00


## Calculate Recency

In [28]:
# Calculate Recency
# Recency is calculated as the number of days between the snapshot date and the customer's last purchase date.
# A smaller Recency value indicates a more recently active customer.

# Group by CustomerID and find the maximum InvoiceDate for each customer (last purchase date)
recency_df = df_cleaned.groupby('Customer ID')['InvoiceDate'].max().reset_index()
recency_df.columns = ['Customer ID', 'LastPurchaseDate']

# Calculate Recency in days
recency_df['Recency'] = (snapshot_date - recency_df['LastPurchaseDate']).dt.days

print("Recency calculation complete. First 5 rows:")
display(recency_df.head())

Recency calculation complete. First 5 rows:


,Customer ID,LastPurchaseDate,Recency
0,12346.0,2011-01-18 10:01:00,326
1,12347.0,2011-12-07 15:52:00,2
2,12348.0,2011-09-25 13:13:00,75
3,12349.0,2011-11-21 09:51:00,19
4,12350.0,2011-02-02 16:01:00,310


In [29]:
recency_df['Recency'].max()

np.int64(739)

In [30]:
recency_df['Recency'].min()

np.int64(1)

In [31]:
recency_df['Recency'].mean()

np.float64(201.331915617557)

In [32]:
recency_df.shape

(5878, 3)

## Calculate Frequency

In [33]:
# Calculate Frequency
# Frequency is the number of unique purchase instances (invoices) for each customer.
# A higher Frequency indicates a more frequent buyer.
# We count unique Invoice numbers for each customer.

frequency_df = df_cleaned.groupby('Customer ID')['Invoice'].nunique().reset_index() # Corrected to InvoiceNo
frequency_df.columns = ['Customer ID', 'Frequency']

print("Frequency calculation complete. First 5 rows:")
display(frequency_df.head())

Frequency calculation complete. First 5 rows:


,Customer ID,Frequency
0,12346.0,12
1,12347.0,8
2,12348.0,5
3,12349.0,4
4,12350.0,1


In [34]:
frequency_df['Frequency'].min()

np.int64(1)

In [37]:
frequency_df['Frequency'].max()

np.int64(398)

In [38]:
frequency_df['Frequency'].mean()

np.float64(6.289384144266758)

In [39]:
frequency_df.shape

(5878, 2)

## Calculate Monetary

In [40]:
# Cell 6: Calculate Monetary
# Monetary is the total amount of money spent by each customer.
# A higher Monetary value indicates a customer who spends more.
# We sum the 'TotalPrice' for each customer, which is derived from Quantity * UnitPrice.

monetary_df = df_cleaned.groupby('Customer ID')['TotalPrice'].sum().reset_index()
monetary_df.columns = ['Customer ID', 'Monetary']

print("Monetary calculation complete. First 5 rows:")
display(monetary_df.head())

Monetary calculation complete. First 5 rows:


,Customer ID,Monetary
0,12346.0,77556.46
1,12347.0,4921.53
2,12348.0,2019.40
3,12349.0,4428.69
4,12350.0,334.40


In [41]:
monetary_df.shape

(5878, 2)

In [42]:
monetary_df['Monetary'].max()

np.float64(580987.04)

In [43]:
monetary_df['Monetary'].min()

np.float64(2.95)

In [44]:
monetary_df['Monetary'].mean()

np.float64(2955.9040945899965)

## Calculate Average Order Value (AOV)

In [47]:
#  Calculate Average Order Value (AOV)
import numpy as np
# AOV = Total Monetary / Total Frequency (Number of Invoices)
# We can use the already calculated Monetary and Frequency
aov_df = pd.merge(monetary_df, frequency_df, on='Customer ID', how='left')
aov_df['AOV'] = aov_df['Monetary'] / aov_df['Frequency']

# Handle potential division by zero if Frequency is 0 (though it shouldn't be with unique InvoiceNo)
aov_df['AOV'] = aov_df['AOV'].replace([np.inf, -np.inf], np.nan).fillna(0) # Replace inf with nan, then nan with 0

aov_df = aov_df[['Customer ID', 'AOV']] # Keep only CustomerID and AOV

print("Average Order Value (AOV) calculation complete. First 5 rows:")
display(aov_df.head())

Average Order Value (AOV) calculation complete. First 5 rows:


,Customer ID,AOV
0,12346.0,6463.038333
1,12347.0,615.191250
2,12348.0,403.880000
3,12349.0,1107.172500
4,12350.0,334.400000


## Number of Unique Products

In [48]:
# Calculate Number of Unique Products
# Number of distinct StockCodes purchased by each customer.
unique_products_df = df_cleaned.groupby('Customer ID')['StockCode'].nunique().reset_index()
unique_products_df.columns = ['Customer ID', 'UniqueProducts']

print("Unique Products purchased calculation complete. First 5 rows:")
display(unique_products_df.head())

Unique Products purchased calculation complete. First 5 rows:


,Customer ID,UniqueProducts
0,12346.0,27
1,12347.0,126
2,12348.0,25
3,12349.0,138
4,12350.0,17


## Calculate Customer Tenure

In [49]:
# Calculate Customer Tenure
# Tenure is the number of days between the snapshot date and the customer's *first* purchase date.
# A longer tenure suggests a more established customer.

first_purchase_df = df_cleaned.groupby('Customer ID')['InvoiceDate'].min().reset_index()
first_purchase_df.columns = ['Customer ID', 'FirstPurchaseDate']

first_purchase_df['Tenure'] = (snapshot_date - first_purchase_df['FirstPurchaseDate']).dt.days

first_purchase_df = first_purchase_df[['Customer ID', 'Tenure']]

print("Customer Tenure calculation complete. First 5 rows:")
display(first_purchase_df.head())

Customer Tenure calculation complete. First 5 rows:


,Customer ID,Tenure
0,12346.0,726
1,12347.0,404
2,12348.0,438
3,12349.0,589
4,12350.0,310


## Process Country Feature (One-Hot Encoding)

In [51]:
# Process Country Feature (One-Hot Encoding)
# We need to get unique countries per customer and then one-hot encode them.
# A customer might only have purchases from one country.

# Get the country for each customer (assuming a customer predominantly buys from one country)
# If a customer bought from multiple countries, this will pick the country of their last transaction.
# For simplicity, let's assume each customer has a primary country associated with their transactions.
# A more robust approach might be to associate the most frequent country or create flags for multi-country buyers.
# For now, let's take the country from their last transaction as their "primary" country.
from sklearn.preprocessing import OneHotEncoder # This is the crucial line
customer_country_df = df_cleaned.groupby('Customer ID')['Country'].last().reset_index()

# Perform One-Hot Encoding on the 'Country' column
# Drop the original 'Country' column after encoding
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_countries = encoder.fit_transform(customer_country_df[['Country']])

# Create a DataFrame from the encoded features
country_names = encoder.get_feature_names_out(['Country'])
country_encoded_df = pd.DataFrame(encoded_countries, columns=country_names, index=customer_country_df.index)

# Add CustomerID back to the encoded DataFrame
country_encoded_df['Customer ID'] = customer_country_df['Customer ID']

print("Country One-Hot Encoding complete. First 5 rows (sample of columns):")
display(country_encoded_df.head())
print(f"Total number of country columns created: {len(country_names)}")

Country One-Hot Encoding complete. First 5 rows (sample of columns):


,Country_Australia,Country_Austria,Country_Bahrain,Country_Belgium,Country_Brazil,Country_Canada,Country_Channel Islands,Country_Cyprus,Country_Czech Republic,Country_Denmark,...,Country_Spain,Country_Sweden,Country_Switzerland,Country_Thailand,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified,Country_West Indies,Customer ID
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,12346.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12347.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12348.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12349.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12350.0


Total number of country columns created: 41


## Combine RFM Features

In [52]:
# Cell 11: Combine All Features into RFM DataFrame
# Start with Recency DataFrame
rfm_df = recency_df[['Customer ID', 'Recency']]

# Merge with Frequency DataFrame
rfm_df = pd.merge(rfm_df, frequency_df, on='Customer ID', how='left')

# Merge with Monetary DataFrame
rfm_df = pd.merge(rfm_df, monetary_df, on='Customer ID', how='left')

# Merge with Average Order Value (AOV) DataFrame
rfm_df = pd.merge(rfm_df, aov_df, on='Customer ID', how='left')

# Merge with Customer Tenure DataFrame
rfm_df = pd.merge(rfm_df, first_purchase_df, on='Customer ID', how='left')

# Merge with Unique Products DataFrame
rfm_df = pd.merge(rfm_df, unique_products_df, on='Customer ID', how='left')

# Merge with One-Hot Encoded Country features
rfm_df = pd.merge(rfm_df, country_encoded_df, on='Customer ID', how='left')

print("All features combined into RFM DataFrame:")
display(rfm_df.head())
print(f"Shape of RFM DataFrame with new features: {rfm_df.shape}")
print("RFM DataFrame Info (showing new columns):")
rfm_df.info()
print("\nDescriptive statistics of RFM features (including new ones):")
display(rfm_df.describe())

All features combined into RFM DataFrame:


,Customer ID,Recency,Frequency,Monetary,AOV,Tenure,UniqueProducts,Country_Australia,Country_Austria,Country_Bahrain,...,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_Thailand,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified,Country_West Indies
0,12346.0,326,12,77556.46,6463.038333,726,27,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,12347.0,2,8,4921.53,615.191250,404,126,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12348.0,75,5,2019.40,403.880000,438,25,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12349.0,19,4,4428.69,1107.172500,589,138,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12350.0,310,1,334.40,334.400000,310,17,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Shape of RFM DataFrame with new features: (5878, 48)
RFM DataFrame Info (showing new columns):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5878 entries, 0 to 5877
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Customer ID                   5878 non-null   float64
 1   Recency                       5878 non-null   int64  
 2   Frequency                     5878 non-null   int64  
 3   Monetary                      5878 non-null   float64
 4   AOV                           5878 non-null   float64
 5   Tenure                        5878 non-null   int64  
 6   UniqueProducts                5878 non-null   int64  
 7   Country_Australia             5878 non-null   float64
 8   Country_Austria               5878 non-null   float64
 9   Country_Bahrain               5878 non-null   float64
 10  Country_Belgium               5878 non-null   float64
 11  Country_Brazil            

,Customer ID,Recency,Frequency,Monetary,AOV,Tenure,UniqueProducts,Country_Australia,Country_Austria,Country_Bahrain,...,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_Thailand,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified,Country_West Indies
count,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,...,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000
mean,15315.313542,201.331916,6.289384,2955.904095,385.180841,474.711637,81.989112,0.002552,0.001871,0.000340,...,0.000170,0.006635,0.003232,0.003403,0.000170,0.001531,0.000681,0.910174,0.001021,0.000170
std,1715.572666,209.338707,13.009406,14440.852688,1214.286459,223.098342,116.484552,0.050456,0.043223,0.018444,...,0.013043,0.081191,0.056767,0.058237,0.013043,0.039103,0.026080,0.285957,0.031936,0.013043
min,12346.000000,1.000000,1.000000,2.950000,2.950000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13833.250000,26.000000,1.000000,342.280000,176.682500,313.000000,19.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,15314.500000,96.000000,3.000000,867.740000,279.242679,530.000000,45.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,16797.750000,380.000000,7.000000,2248.305000,414.902458,668.000000,103.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,18287.000000,739.000000,398.000000,580987.040000,84236.250000,739.000000,2550.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [53]:
rfm_df.shape,rfm_df.size,rfm_df.columns

((5878, 48),
 282144,
 Index(['Customer ID', 'Recency', 'Frequency', 'Monetary', 'AOV', 'Tenure',
        'UniqueProducts', 'Country_Australia', 'Country_Austria',
        'Country_Bahrain', 'Country_Belgium', 'Country_Brazil',
        'Country_Canada', 'Country_Channel Islands', 'Country_Cyprus',
        'Country_Czech Republic', 'Country_Denmark', 'Country_EIRE',
        'Country_European Community', 'Country_Finland', 'Country_France',
        'Country_Germany', 'Country_Greece', 'Country_Iceland',
        'Country_Israel', 'Country_Italy', 'Country_Japan', 'Country_Korea',
        'Country_Lebanon', 'Country_Lithuania', 'Country_Malta',
        'Country_Netherlands', 'Country_Nigeria', 'Country_Norway',
        'Country_Poland', 'Country_Portugal', 'Country_RSA',
        'Country_Saudi Arabia', 'Country_Singapore', 'Country_Spain',
        'Country_Sweden', 'Country_Switzerland', 'Country_Thailand',
        'Country_USA', 'Country_United Arab Emirates', 'Country_United Kingdom',
  

In [55]:
# Cell 12: Save RFM Features
# Define the path to save the final RFM features file
# Since this notebook might be in 'models' folder, output to '../data'
output_data_folder = '../data'
output_file_name = 'rfm_features_enhanced.xlsx' # Changed filename to indicate enhanced features
output_file_path = os.path.join(output_data_folder, output_file_name)

print(f"Attempting to save enhanced RFM features to: {output_file_path}")

rfm_df.to_excel(output_file_path, index=False)

print(f"Enhanced RFM features successfully saved to '{output_file_path}'.")

Attempting to save enhanced RFM features to: ../data\rfm_features_enhanced.xlsx
Enhanced RFM features successfully saved to '../data\rfm_features_enhanced.xlsx'.
